In [4]:
import pandas as pd
pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint, LineString, LinearRing
from shapely.ops import cascaded_union, unary_union, transform
import datetime
import math
from scipy.ndimage.interpolation import shift
import scipy.interpolate as si
import shapely.wkt
from shapely.validation import explain_validity
import xarray as xr
import seaborn as sns
from my_functions import sat_vap_press, vap_press, hot_dry_windy, haines
from joblib import Parallel, delayed
import multiprocessing
from os.path import exists
import rasterio
from rasterio.windows import get_data_window,Window, from_bounds
from rasterio.plot import show
from itertools import product

from timezonefinder import TimezoneFinder
import pytz

from helper_functions import build_one_gridcell, calculate_intersection, calculate_grid_cell_corners, make_file_namelist, generate_df

## Unweighted Fire Features
We are testing the effect of taking the non-area weighted mean, ie the mean over the intersecting grid cells, for the April 17th 2023 research update slideshow

In [19]:
fire_incidents = ['204 COW', 'GRANITE GULCH', 'SHADY', 'WILLIAMS FLATS', 'PEDRO MOUNTAIN', 'WALKER',
                  'AUGUST COMPLEX', 'BOBCAT', 'CAMERON PEAK', 'CREEK', 'DOLAN', 'EAST TROUBLESOME',
                  'HOLIDAY FARM', 'LAKE','RIVERSIDE', 'PINE GULCH', 'CZU', 'SCU']
"""fire_incidents = ['AUGUST COMPLEX', 'BOBCAT', 'CAMERON PEAK', 'CREEK', 'DOLAN', 'EAST TROUBLESOME',
                  'HOLIDAY FARM', 'LAKE','RIVERSIDE', 'PINE GULCH', 'CZU', 'SCU']"""


"""fire_incidents=['BOBCAT', 'CAMERON PEAK', 'CREEK','DOLAN', 'EAST TROUBLESOME',
                  'HOLIDAY FARM', 'LAKE','RIVERSIDE', 'PINE GULCH', 'CZU', 'SCU']"""

#fire_incidents=['AUGUST COMPLEX']
#fire_incidents=['BOBCAT']
fire_incidents=['LAKE']
#fire_incidents = ['GRANITE GULCH', 'SHADY', 'WILLIAMS FLATS', 'PEDRO MOUNTAIN', 'WALKER']

path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = 'Z_day_start.geojson'
start_time=12

for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    print(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily = gpd.read_file(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    print(fire_daily.crs)
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares. from m2
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    print(fire_daily)
    
    #THE TOP FEATURES ARE HWP, PWS, ESI, LOW FUEL, DROUGHT CODE, BLENDED_SM
    
    """#HRRR
    hrrr_unweighted = hrrr_timeseries(fire_daily, start_time, False)
    print(hrrr_unweighted)
    hrrr_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_HRRR_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #pws
    pws_unweighted=pws_timeseries(fire_daily,start_time, False)
    print(pws_unweighted)
    pws_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_PWS_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #esi
    esi_unweighted = esi_timeseries(fire_daily, start_time, False)
    print(esi_unweighted)
    esi_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_ESI_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #fuel loading (1km)
    fuel_unweighted = fuel_loading_timeseries(fire_daily, start_time, False)
    print(fuel_unweighted)
    fuel_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_FUEL_LOADING_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #imerg 
    imerg_unweighted = imerg_fwi_timeseries(fire_daily, start_time, False)
    print(imerg_unweighted)
    imerg_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_FWI_IMERG_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    
    #smops
    smops_unweighted=smops_timeseries(fire_daily,start_time, False)
    print(smops_unweighted)
    smops_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_SMOPS_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    """
   
    """hdw = hdw_timeseries(fire_daily, start_time, False)
    print(hdw)
    hdw.to_csv('./fire_features_3/'+fire_name+'_Daily_Lagged_HDW_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    """
    

/data2/lthapa/ML_daily/fire_polygons/lake_VIIRS_daily_12Z_day_start.geojson
epsg:3347
   12Z Start Day  Incident Number Fire Name     UTC Day   Lat Fire  \
0     2020-08-12       11773470.0      LAKE  2020-08-13  34.678611   
1     2020-08-13       11773470.0      LAKE  2020-08-14  34.678611   
2     2020-08-14       11773470.0      LAKE  2020-08-15  34.678611   
3     2020-08-15       11773470.0      LAKE  2020-08-16  34.678611   
4     2020-08-16       11773470.0      LAKE  2020-08-17  34.678611   
5     2020-08-17       11773470.0      LAKE  2020-08-18  34.678611   
6     2020-08-18       11773470.0      LAKE  2020-08-19  34.678611   
7     2020-08-19       11773470.0      LAKE  2020-08-20  34.678611   
8     2020-08-20       11773470.0      LAKE  2020-08-21  34.678611   
9     2020-08-21       11773470.0      LAKE  2020-08-22  34.678611   
10    2020-08-22       11773470.0      LAKE  2020-08-23  34.678611   
11    2020-08-29       11773470.0      LAKE  2020-08-30  34.678611   

   

## Weighted Fire Features
We have been weighting the features by the area of the polygon contained in each intersecting grid cell, and this was used exclusively in the machine learning and index building until the April 10th, 2023 research Update slideshow

In [73]:
path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = 'Z_Day_Start.geojson'
start_time=12
resources_file  = '../YYYY_PCT_CONT.xlsx'

#years =[2019,2020]
years = [2020]



for jj in range(len(years)):
    print(path_poly+'ClippedFires'+str(years[jj])+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    
    fire_daily = gpd.read_file(path_poly+'ClippedFires'+str(years[jj])+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    print(fire_daily.crs)
    
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares. from m2
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
    irwinIDs = np.unique(fire_daily['irwinID'].values)
    print('We are processing ' +str(len(irwinIDs)) + ' unique fires for '+ str(years[jj]))

 
    
    for ii in range(len(irwinIDs)):
        print(ii)
        fireID=irwinIDs[ii]
        print(fireID)
        df_fire = fire_daily[fire_daily['irwinID']==fireID] #this is what gets fed to the feature selection code

        #maybe we filter 12Z Start dates to where we have data?
        days=np.array(df_fire['12Z Start Day'].values, dtype='datetime64')
        df_fire = df_fire[(days>=np.datetime64(str(years[jj])+'-07-01'))&
                              (days<=np.datetime64(str(years[jj]+1)+'-01-01'))].reset_index(drop=True)
        
        ncar_weighted, ncar_unweighted = ncar_timeseries(df_fire, start_time)
        ncar_weighted = pd.concat([esi_weighted, pd.DataFrame({'irwinID':[fireID]*len(esi_weighted)})], axis=1)
        ncar_unweighted = pd.concat([esi_unweighted, pd.DataFrame({'irwinID':[fireID]*len(esi_unweighted)})], axis=1)
        print(ncar_weighted)
        print(ncar_unweighted)
        
        ncar_weighted.to_csv('./fire_features_ncar/'+fireID+str(years[jj])+'_Daily_ESI_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
        ncar_unweighted.to_csv('./fire_features_ncar/'+fireID+str(years[jj])+'_Daily_ESI_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
        q
    


/data2/lthapa/ML_daily/fire_polygons/ClippedFires2020_VIIRS_daily_12Z_Day_Start.geojson
epsg:3347
We are processing 273 unique fires for 2020
   INC209R_IDENTIFIER     REPORT_FROM_DATE       REPORT_TO_DATE  \
0            11683732  01/09/2020 12:30:00  01/10/2020 08:15:00   
1            11683744  01/10/2020 08:00:00  01/12/2020 08:00:00   
2            11683755  01/10/2020 09:00:00  01/11/2020 09:00:00   
3            11683718  01/08/2020 12:30:00  01/08/2020 21:15:00   
4            11683871  01/09/2020 17:00:00  01/10/2020 14:00:00   

   PCT_CONTAINED_COMPLETED  INCIDENT_NAME  \
0                     96.0  Dogwood Trail   
1                    100.0  Dogwood Trail   
2                    100.0  Dogwood Trail   
3                     90.0  Dogwood Trail   
4                    100.0       WEST 380   

                       IRWIN_IDENTIFIER  
0  AA302C33-2DC5-44D0-9A55-19A133EF04B3  
1  AA302C33-2DC5-44D0-9A55-19A133EF04B3  
2  AA302C33-2DC5-44D0-9A55-19A133EF04B3  
3  AA302C33-2DC5

TypeError: cannot unpack non-iterable NoneType object

In [95]:
print(df_fire)
ncar_weighted, ncar_unweighted = ncar_timeseries(df_fire, start_time)
print(ncar_weighted)

   12Z Start Day                               irwinID  Incident Number  \
0     2020-09-07  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
1     2020-09-08  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
2     2020-09-09  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
3     2020-09-10  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
4     2020-09-11  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
5     2020-09-12  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
6     2020-09-13  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
7     2020-09-14  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
8     2020-09-15  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
9     2020-09-16  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
10    2020-09-17  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
11    2020-09-19  00B3EDB2-87A4-4D0A-A834-4E1012BF9693       11943503.0   
12    2020-09-20  00B3EDB

In [94]:
#ncar grid was 1000m = 1km = grid resolution
# fwi grid was 10000m=10km = grid resolution
def ncar_timeseries(df, day_start_hour):
    varis_ncar = ['day','FMCG2D','FMCGLH2D']
    df_ncar_weighted = generate_df(varis_ncar, len(df))
    df_ncar_unweighted = generate_df(varis_ncar, len(df))

    #do the intersection, in parallel
    ncar_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'NCAR_MOISTURE_GRID',1000) 
                                 for ii in range(len(df)))
    print([ncar_intersections[jj]['weights'].sum() for jj in range(len(ncar_intersections))])

    
    fire_ncar_intersection=gpd.GeoDataFrame(pd.concat(ncar_intersections, ignore_index=True))
    fire_ncar_intersection.set_geometry(col='geometry')  
    fire_ncar_intersection = fire_ncar_intersection.set_index([str(day_start_hour)+'Z Start Day', 'row', 'col'])
    fire_ncar_intersection=fire_ncar_intersection[~fire_ncar_intersection.index.duplicated()]

    fire_ncar_intersection_xr = fire_ncar_intersection.to_xarray()
    fire_ncar_intersection_xr['weights_mask'] = xr.where(fire_ncar_intersection_xr['weights']>0,1, np.nan)
    print(fire_ncar_intersection_xr)
    #print(fire_ncar_intersection)
    #load in rave data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    ncar_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/YYYY/FMC/fmc_YYYYMMDD_20Z.nc')

    
    dat_ncar = xr.open_mfdataset(ncar_filenames,concat_dim='Time',combine='nested',compat='override', coords='all')
    #dat_ncar = dat_ncar.assign_coords({'Time': times_back_used}) #assign coords so we can select in time
    dat_ncar = dat_ncar.reindex(Time=times_back_used,method='nearest') #makes the weekly data daily
    print(dat_ncar)
    
    
    #select the locations and times we want
    dat_ncar_sub = dat_ncar.isel(south_north = fire_ncar_intersection_xr['row'].values.astype(int), 
                                 west_east = fire_ncar_intersection_xr['col'].values.astype(int)).sel(
                                 Time = pd.to_datetime(fire_ncar_intersection_xr[str(day_start_hour)+'Z Start Day'].values))#these should be lined up correctly
    print(dat_ncar_sub['FMCG2D'])
    df_ncar_weighted['day'].iloc[:] = pd.to_datetime(fire_ncar_intersection_xr[str(day_start_hour)+'Z Start Day'].values)
    df_ncar_unweighted['day'].iloc[:] = pd.to_datetime(fire_ncar_intersection_xr[str(day_start_hour)+'Z Start Day'].values)

    for var in varis_ncar[1:]:
        df_ncar_weighted[var] = np.nansum(fire_ncar_intersection_xr['weights'].values*dat_ncar_sub[var].values, axis=(1,2))
        df_ncar_unweighted[var] = np.nansum(fire_ncar_intersection_xr['weights'].values*dat_ncar_sub[var].values, axis=(1,2))

           
    #this day is messed up, fill it in with NANS
    df_ncar_weighted.iloc[df_ncar_weighted['day']=='2020-09-09'] = [pd.date_range(np.datetime64('2020-09-09'),np.datetime64('2020-09-09')+np.timedelta64(0,'D')),
                                                           np.nan,np.nan]
    df_ncar_unweighted.iloc[df_ncar_unweighted['day']=='2020-09-09'] = [pd.date_range(np.datetime64('2020-09-09'),np.datetime64('2020-09-09')+np.timedelta64(0,'D')),
                                                           np.nan,np.nan]
    return df_ncar_weighted, df_ncar_unweighted

## test out other features

In [ ]:
      print(ii)
        fireID=irwinIDs[ii]
        print(fireID)
        df_fire = fire_daily[fire_daily['irwinID']==fireID] #this is what gets fed to the feature selection code
        
        #maybe we filter 12Z Start dates to where we have data?
        days=np.array(df_fire['12Z Start Day'].values, dtype='datetime64')
        df_fire = df_fire[(days>=np.datetime64(str(years[jj])+'-07-01'))&
                              (days<=np.datetime64(str(years[jj]+1)+'-01-01'))].reset_index(drop=True)

        #PWS
        esi_weighted, esi_unweighted = esi_timeseries(df_fire, start_time)
        esi_weighted = pd.concat([esi_weighted, pd.DataFrame({'irwinID':[fireID]*len(esi_weighted)})], axis=1)
        esi_unweighted = pd.concat([esi_unweighted, pd.DataFrame({'irwinID':[fireID]*len(esi_unweighted)})], axis=1)
        print(esi_weighted)
        print(esi_unweighted) loading_weighted_all = pd.DataFrame() 
    loading_unweighted_all = pd.DataFrame()
    
    for ii in range(0,1):#len(irwinIDs)):
        print(ii)
        fireID=irwinIDs[ii]
        print(fireID)
        df_fire = fire_daily[fire_daily['irwinID']==fireID] #this is what gets fed to the feature selection code
        
        #maybe we filter 12Z Start dates to where we have data?
        days=np.array(df_fire['12Z Start Day'].values, dtype='datetime64')
        df_fire = df_fire[(days>=np.datetime64(str(years[jj])+'-07-01'))&
                              (days<=np.datetime64(str(years[jj]+1)+'-01-01'))].reset_index(drop=True)

        #PWS
        loading_weighted, loading_unweighted = fuel_loading_timeseries(df_fire, start_time)
        loading_weighted = pd.concat([loading_weighted, pd.DataFrame({'irwinID':[fireID]*len(loading_weighted)})], axis=1)
        loading_unweighted = pd.concat([loading_unweighted, pd.DataFrame({'irwinID':[fireID]*len(loading_unweighted)})], axis=1)
        print(loading_weighted)
        print(loading_unweighted)
        
        loading_weighted_all = pd.concat([loading_weighted_all, loading_weighted], axis=0).reset_index(drop=True)
        loading_unweighted_all = pd.concat([loading_unweighted_all, loading_unweighted], axis=0).reset_index(drop=True)
    
    loading_weighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_LOADING_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    loading_unweighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_LOADING_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages"""#NOAA Soil Moisture
        #smops = smops_timeseries(df_fire,start_time, False) #UNWEIGHTED
        #smops = smops_timeseries(df_fire,start_time, True) #WEIGHTED
        #smops = pd.concat([smops, pd.DataFrame({'irwinID':[fireID]*len(smops)})], axis=1)
        #print(smops_weighted)
        #smops_all = pd.concat([smops_all, smops], axis=0).reset_index(drop=True)
        
        #RAVE (always unweighted SUM)
        """rave = rave_timeseries(df_fire, start_time, 24, True)
        rave = pd.concat([rave, pd.DataFrame({'irwinID':[fireID]*len(rave)})], axis=1)
        print(rave)
        rave_all = pd.concat([rave_all, rave], axis=0).reset_index(drop=True)"""
        
        #HRRR HWP
        """hwp_weighted, hwp_unweighted = hwp_timeseries(df_fire, start_time)
        hwp_weighted = pd.concat([hwp_weighted, pd.DataFrame({'irwinID':[fireID]*len(hwp_weighted)})], axis=1)
        hwp_unweighted = pd.concat([hwp_unweighted, pd.DataFrame({'irwinID':[fireID]*len(hwp_unweighted)})], axis=1)
        print(hwp_weighted)
        print(hwp_unweighted)
        hwp_weighted_all = pd.concat([hwp_weighted_all, hwp_weighted], axis=0).reset_index(drop=True)
        hwp_unweighted_all = pd.concat([hwp_unweighted_all, hwp_unweighted], axis=0).reset_index(drop=True)"""
        
        #HRRR HDW
        hdw_weighted, hdw_unweighted = hdw_lagged_timeseries(df_fire, start_time)
        hdw_weighted = pd.concat([hdw_weighted, pd.DataFrame({'irwinID':[fireID]*len(hdw_weighted)})], axis=1)
        hdw_unweighted = pd.concat([hdw_unweighted, pd.DataFrame({'irwinID':[fireID]*len(hdw_unweighted)})], axis=1)
        print(hdw_weighted)
        print(hdw_unweighted)
        
        hdw_weighted_all = pd.concat([hdw_weighted_all, hdw_weighted], axis=0).reset_index(drop=True)
        hdw_unweighted_all = pd.concat([hdw_unweighted_all, hdw_unweighted], axis=0).reset_index(drop=True)
    
    hdw_weighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_HDW_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    hdw_unweighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_HDW_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily average        
    
    print(ii)
        fireID=irwinIDs[ii]
        print(fireID)
        df_fire = fire_daily[fire_daily['irwinID']==fireID] #this is what gets fed to the feature selection code
        
        #maybe we filter 12Z Start dates to where we have data?
        days=np.array(df_fire['12Z Start Day'].values, dtype='datetime64')
        df_fire = df_fire[(days>=np.datetime64(str(years[jj])+'-07-01'))&
                              (days<=np.datetime64(str(years[jj]+1)+'-01-01'))].reset_index(drop=True)
        #df_fire.to_crs('EPSG:4326').plot()
        #HRRR HDW
        imerg_weighted, imerg_unweighted = imerg_fwi_timeseries(df_fire, start_time)
        imerg_weighted = pd.concat([imerg_weighted, pd.DataFrame({'irwinID':[fireID]*len(imerg_weighted)})], axis=1)
        imerg_unweighted = pd.concat([imerg_unweighted, pd.DataFrame({'irwinID':[fireID]*len(imerg_unweighted)})], axis=1)
        print(imerg_weighted)
        print(imerg_unweighted)
        
        imerg_weighted_all = pd.concat([imerg_weighted_all, imerg_weighted], axis=0).reset_index(drop=True)
        imerg_unweighted_all = pd.concat([imerg_unweighted_all, imerg_unweighted], axis=0).reset_index(drop=True)
        #PWS
        pws_weighted, pws_unweighted = pws_timeseries(df_fire, start_time)
        pws_weighted = pd.concat([pws_weighted, pd.DataFrame({'irwinID':[fireID]*len(pws_weighted)})], axis=1)
        pws_unweighted = pd.concat([pws_unweighted, pd.DataFrame({'irwinID':[fireID]*len(pws_unweighted)})], axis=1)
        print(pws_weighted)
        print(pws_unweighted)
        
        pws_weighted_all = pd.concat([pws_weighted_all, pws_weighted], axis=0).reset_index(drop=True)
        pws_unweighted_all = pd.concat([pws_unweighted_all, pws_unweighted], axis=0).reset_index(drop=True)
    
    pws_weighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_PWS_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    pws_unweighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_PWS_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #print(imerg_weighted_all)
    #imerg_weighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_IMERG_FWI_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    #imerg_unweighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_IMERG_FWI_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily average        
    #print(rave_all)
    #rave_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_RAVE_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily sums
    #smops_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_SMOPS_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    #hwp_weighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_HWP_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    #hwp_unweighted_all.to_csv('./fire_features_3/'+'ClippedFires'+str(years[jj])+'_Daily_HWP_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages

## Call other feature extraction functions

In [ ]:
 #rave
    """rave_weighted = rave_timeseries(fire_daily, start_time, 24, True)
    print(rave_weighted)
    rave_weighted.to_csv('./fire_features_3/'+fire_name+'_Daily_RAVE_Weighted_'+str(start_time)+'Z_day_start.csv')
    
    rave_unweighted = rave_timeseries(fire_daily, start_time, 24, False)
    print(rave_unweighted)
    rave_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_RAVE_Unweighted_'+str(start_time)+'Z_day_start.csv')
    """
    
    #smops
    """smops_unweighted=smops_timeseries(fire_daily,start_time, False)
    print(smops_unweighted)
    smops_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_SMOPS_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    """
    
    #pws
    """pws_weighted=pws_timeseries(fire_daily,start_time, True)
    print(pws_weighted)
    pws_weighted.to_csv('./fire_features_3/'+fire_name+'_Daily_PWS_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    pws_unweighted=pws_timeseries(fire_daily,start_time, False)
    print(pws_unweighted)
    pws_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_PWS_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages"""
    
    
    #imerg 
    """imerg_weighted = imerg_fwi_timeseries(fire_daily, start_time, True)
    print(imerg_weighted)
    imerg_weighted.to_csv('./fire_features_3/'+fire_name+'_Daily_FWI_IMERG_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    imerg_unweighted = imerg_fwi_timeseries(fire_daily, start_time, False)
    print(imerg_unweighted)
    imerg_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_FWI_IMERG_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    """
    
    #fuel loading (1km)
    """fuel_weighted = fuel_loading_timeseries(fire_daily, start_time, True)
    print(fuel_weighted)
    fuel_weighted.to_csv('./fire_features_3/'+fire_name+'_Daily_FUEL_LOADING_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    fuel_unweighted = fuel_loading_timeseries(fire_daily, start_time, False)
    print(fuel_unweighted)
    fuel_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_FUEL_LOADING_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    """
    #esi
    """esi_weighted = esi_timeseries(fire_daily, start_time, True)
    print(esi_weighted)
    esi_weighted.to_csv('./fire_features_3/'+fire_name+'_Daily_ESI_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    esi_unweighted = esi_timeseries(fire_daily, start_time, False)
    print(esi_unweighted)
    esi_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_ESI_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages"""
    
    #HRRR
    #hrrr_weighted = hrrr_timeseries(fire_daily, start_time, True)
    #print(hrrr_weighted)
    #hrrr_weighted.to_csv('./fire_features_3/'+fire_name+'_Daily_HRRR_Weighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #HDW with lags
    """hdw = hdw_timeseries(fire_daily, start_time, True)
    print(hdw)
    hdw.to_csv('./fire_features_3/'+fire_name+'_Daily_Lagged_HDW_'+str(start_time)+'Z_day_start.csv') #daily averages
    """
    
    """
    hrrr_unweighted = hrrr_timeseries(fire_daily, start_time, False)
    print(hrrr_unweighted)
    hrrr_unweighted.to_csv('./fire_features_3/'+fire_name+'_Daily_HRRR_Unweighted_'+str(start_time)+'Z_day_start.csv') #daily averages
    """
    
    #sit_209
    
    """ sit209 = resources_timeseries(fire_daily, start_time)
    print(sit209)
    sit209.to_csv('./fire_features_3/'+fire_name+'_Daily_RESOURCES_'+str(start_time)+'Z_day_start.csv') #daily averages"""


### Fill in Missing Data
1. 11/24 in the Rave Data

In [ ]:
data = xr.open_dataset('/data2/lthapa/2020/RAVE/11/RAVE-HrlyEmiss-3km-CONUS_v1r1_blend_s20201125.nc')

times = data.time.values
times_shifted = times-np.timedelta64(1,'D')
data['time'] = times_shifted
data.assign_coords({'time':times_shifted})
print(data.time.values)
#print(data.variables)
varis = ['FRP_MEAN', 'FRP_SD', 'FRE', 'CO2', 'CO', 'SO2', 'OC','BC', 'PM25', 'NOx', 'NH3','TPM', 'VOCs', 'CH4']
for var in varis:
    data[var].where(data[var] != np.nan)
    
#print(data.variables)

In [ ]:
data.dimensions

In [ ]:
data.to_netcdf('/data2/lthapa/2020/RAVE/11/RAVE-HrlyEmiss-3km-CONUS_v1r1_blend_s20201124.nc')

In [ ]:
da = xr.DataArray(np.arange(16).reshape(4, 4), dims=["x", "y"])
da

In [ ]:
da.where(da.x + da.y < 4)

In [ ]:
fig, ax = plt.subplots(figsize=(20,15)) 
rave.plot(x='day', y=['FRP_MEAN'],
                                figsize=(20,6),style='ro-',ax=ax )
h1, l1 = ax.get_legend_handles_labels()
ax.legend(h1, l1,bbox_to_anchor=(1.1, 1), loc='upper left')
plt.title('LAKE FRP')
plt.show()

## Dataset-Dependent Helper Functions

### SIT-209 Data

In [59]:
def resources_timeseries(df,day_start_hour, sit209_data):
    #sit209_data = pd.read_csv('../merged_sit.csv')
    
    
    #get the fire incident number, lat, and lon
    incident_number = df['irwinID'].iloc[0]
    fire_lat = df['Lat Fire'].iloc[0]
    fire_lon = df['Lon Fire'].iloc[0]
    #print(incident_number, fire_lat, fire_lon)
    
    sit209_data_fire = sit209_data[sit209_data['IRWIN_IDENTIFIER']==incident_number]
    print(sit209_data_fire)
    #print(sit209_data_fire.columns.values)
    #do the time zone conversion
    obj=TimezoneFinder() #initialize the timezone finder
    tz = obj.timezone_at(lng=fire_lon, lat=fire_lat) #get the timezone
    local = pytz.timezone(tz)
    utc = pytz.utc
    
    #put the start and end times in local time
    loc_dt_start = [local.localize(datetime.datetime.strptime(date, '%m/%d/%Y %H:%M:%S')) for date in sit209_data_fire['REPORT_FROM_DATE'].values]
    loc_dt_end = [local.localize(datetime.datetime.strptime(date, '%m/%d/%Y %H:%M:%S')) for date in sit209_data_fire['REPORT_TO_DATE'].values]
    
    #put them in UTC time
    utc_dt_start = [time_start.astimezone(utc) for time_start in loc_dt_start]
    utc_dt_end = [time_end.astimezone(utc) for time_end in loc_dt_end]
    
    start_day = pd.to_datetime(utc_dt_start[0]).strftime('%Y-%m-%d')+' '+str(day_start_hour)+':00'
    
    
    #reassign to UTC time, this DOES keep track of daylight savings (eg +7 is used for PDT, +8 is used for PST)
    sit209_data_fire['Report Start UTC'] = pd.to_datetime(utc_dt_start)
    sit209_data_fire['Report End UTC'] = pd.to_datetime(utc_dt_end)
    sit209_data_fire['Timezone']= tz
    
    #localise the index
    sit209_data_fire = sit209_data_fire.set_index(['Report Start UTC']).tz_localize(None)
    #print(sit209_data_fire.iloc[0:4])
    
    
    ## do the 12z-12z day grouping, based on the UTC times
    #start_day_utc = str(utc_dt_start[0])
    start_day_utc=str(df[str(day_start_hour)+'Z Start Day'][0])
    start_datetime_utc = np.datetime64(start_day_utc[0:10]+'T'+str(day_start_hour).zfill(2)+':00')
    print(start_datetime_utc)
    #sit209_data_fire = sit209_data_fire.resample('24H',origin=start_datetime_utc)

    #personnel = sit209_data_fire['RESOURCE_PERSONNEL'].resample('24H',origin=start_datetime_utc).sum().reset_index()
    percent_contained = sit209_data_fire['PCT_CONTAINED_COMPLETED'].resample('24H',origin=start_datetime_utc).mean().reset_index()
    df_sit209 = pd.concat([percent_contained],axis=1)
    df_sit209.columns=['day', 'percent_contained']
    df_sit209['day'] = pd.to_datetime(df_sit209['day'].values).strftime('%Y-%m-%d')
    df_sit209=df_sit209.fillna(method='ffill')
    #df_sit209['day'].iloc[:] = pd.to_datetime(df[str(day_start_hour)+ 'Z Start Day'].values)

    #inds = df_sit209['day'].isin(df[str(day_start_hour)+'Z Start Day']).values
    
    return df_sit209
    

### HRRR HWP

In [84]:
#HRRR_WS formulation from, take time mean, then take weighted average. For HDW, multiply the weighted means of VPD and WIND
def hwp_timeseries(df,day_start_hour):  #with the wind speed
    varis_hrrr_derived = ['day','hwp'] #'hd1w0','hd2w0', 'hd3w0', 'hd4w0', 'hd5w0',
    
    #return both!
    df_hwp_weighted = generate_df(varis_hrrr_derived, len(df))
    df_hwp_unweighted = generate_df(varis_hrrr_derived, len(df))

    #do the intersection, in parallel
    #print(tic)
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'HRRR_GRID',3000) 
                                 for ii in range(len(df)))
    print([hrrr_intersections[jj]['weights'].sum() for jj in range(len(hrrr_intersections))])

    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        intersection_sub['weights_mask'] =xr.where(intersection_sub['weights']>0,1, np.nan)
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(1,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        #print(times_back)
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/processed_hrrr_hdw_hwp/Processed_HRRR_YYYYMMDDHH_HDW_HWP.nc')
        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})
        
        dat_hrrr['hd0w0'] = dat_hrrr['wind_speed']*dat_hrrr['vpd_2m']
        #print(dat_hrrr)
        
        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)) #get the location of the overlaps
    
        df_hwp_weighted['day'].iloc[count] =today# pd.to_datetime([str(day_start_hour)+ 'Z Start Day'].values[count])            df_hrrr_derived.loc[count, ('hd0w0')] = np.nansum((hrrr_daily_mean_region['hd0w0'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        df_hwp_unweighted['day'].iloc[count] =today# pd.to_datetime([str(day_start_hour)+ 'Z Start Day'].values[count])            df_hrrr_derived.loc[count, ('hd0w0')] = np.nansum((hrrr_daily_mean_region['hd0w0'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))

        df_hwp_weighted.loc[count, ('hwp')] =np.nansum((hrrr_daily_mean_region['hwp'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))   
        df_hwp_unweighted.loc[count, ('hwp')] = np.nanmean((hrrr_daily_mean_region['hwp'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights_mask'].values))

        dat_hrrr.close()
        count =count+1
        #print(df_hrrr_derived)
    return df_hwp_weighted, df_hwp_unweighted

### HRRR HDW

In [101]:
#HRRR_WS formulation from, take time mean, then take weighted average. For HDW, multiply the weighted means of VPD and WIND
def hdw_lagged_timeseries(df,day_start_hour):  #with the wind speed
    varis_hrrr_derived = ['day','hd0w0', 'hd1w0','hd2w0', 'hd3w0']#, 'hd4w0', 'hd5w0',
    df_hdw_weighted = generate_df(varis_hrrr_derived, len(df))
    df_hdw_unweighted = generate_df(varis_hrrr_derived, len(df))
    
    #do the intersection, in parallel
    #print(tic)
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'HRRR_GRID',3000) 
                                 for ii in range(len(df)))
    
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        intersection_sub['weights_mask'] =xr.where(intersection_sub['weights']>0,1, np.nan)
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(3,'D'), end=np.datetime64(today)+
                                   np.timedelta64(36,'h'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/processed_hrrr_hdw_hwp/Processed_HRRR_YYYYMMDDHH_HDW_HWP.nc')
        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})
        dat_hrrr = dat_hrrr.resample(time='h').asfreq()
        #print(dat_hrrr['time'].values)
        #define the days
        day0= np.datetime64(today)+np.timedelta64(12,'h')
        day1 =day0-np.timedelta64(1,'D')
        day2 =day0-np.timedelta64(2,'D')
        day3 =day0-np.timedelta64(3,'D')

        #define the times we will select for VPD
        times_0 = pd.date_range(start=day0, end=day0+np.timedelta64(23,'h'),freq='H')
        times_1 = pd.date_range(start=day1, end=day1+np.timedelta64(23,'h'),freq='H')
        times_2 = pd.date_range(start=day2, end=day2+np.timedelta64(23,'h'),freq='H')
        times_3 = pd.date_range(start=day3, end=day3+np.timedelta64(23,'h'),freq='H')
        
        w0 = dat_hrrr['wind_speed'].sel(time=times_0, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd0 = dat_hrrr['vpd_2m'].sel(time=times_0, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd1 = dat_hrrr['vpd_2m'].sel(time=times_1, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd1=hd1.assign_coords({'time':w0['time'].values})
        hd2 = dat_hrrr['vpd_2m'].sel(time=times_2, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd2=hd2.assign_coords({'time':w0['time'].values})
        hd3 = dat_hrrr['vpd_2m'].sel(time=times_3, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd3=hd3.assign_coords({'time':w0['time'].values})
            
        hd0w0 = hd0*w0
        hd1w0 = hd1*w0
        hd2w0 = hd2*w0
        hd3w0 = hd3*w0

        hd0w0_daily_mean = hd0w0.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        hd1w0_daily_mean = hd1w0.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        hd2w0_daily_mean = hd2w0.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        hd3w0_daily_mean = hd3w0.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        

        
        df_hdw_weighted['day'].iloc[count] =today# pd.to_datetime([str(day_start_hour)+ 'Z Start Day'].values[count])
        df_hdw_unweighted['day'].iloc[count] =today# pd.to_datetime([str(day_start_hour)+ 'Z Start Day'].values[count])

        #WEIGHTED
        df_hdw_weighted.loc[count, ('hd0w0')] = np.nansum((hd0w0_daily_mean.values)*(intersection_sub['weights'].values))
        df_hdw_weighted.loc[count, ('hd1w0')] = np.nansum((hd1w0_daily_mean.values)*(intersection_sub['weights'].values))
        df_hdw_weighted.loc[count, ('hd2w0')] = np.nansum((hd2w0_daily_mean.values)*(intersection_sub['weights'].values))
        df_hdw_weighted.loc[count, ('hd3w0')] = np.nansum((hd3w0_daily_mean.values)*(intersection_sub['weights'].values))

        #UNWEIGHTED
        df_hdw_unweighted.loc[count, ('hd0w0')] = np.nanmean((hd0w0_daily_mean.values))
        df_hdw_unweighted.loc[count, ('hd1w0')] = np.nanmean((hd1w0_daily_mean.values))
        df_hdw_unweighted.loc[count, ('hd2w0')] = np.nanmean((hd2w0_daily_mean.values))
        df_hdw_unweighted.loc[count, ('hd3w0')] = np.nanmean((hd3w0_daily_mean.values))
        
        count=count+1
    return df_hdw_weighted, df_hdw_unweighted

### HRRR HDW and HWP

In [17]:
#HRRR_WS formulation from, take time mean, then take weighted average. For HDW, multiply the weighted means of VPD and WIND
def hdw_timeseries(df,day_start_hour, use_weights):  #with the wind speed
    varis_hrrr_derived = ['day','hd0w0', 'hd1w0','hd2w0', 'hd3w0']#, 'hd4w0', 'hd5w0',
    df_hrrr_derived = generate_df(varis_hrrr_derived, len(df))
    
    #do the intersection, in parallel
    #print(tic)
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'HRRR_GRID',3000) 
                                 for ii in range(len(df)))
    
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        intersection_sub['weights_mask'] =xr.where(intersection_sub['weights']>0,1, np.nan)
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(3,'D'), end=np.datetime64(today)+
                                   np.timedelta64(36,'h'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/processed_hrrr_hdw_hwp/Processed_HRRR_YYYYMMDDHH_HDW_HWP.nc')
        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})
        dat_hrrr = dat_hrrr.resample(time='h').asfreq()
        print(dat_hrrr['time'].values)
        #define the days
        day0= np.datetime64(today)+np.timedelta64(12,'h')
        day1 =day0-np.timedelta64(1,'D')
        day2 =day0-np.timedelta64(2,'D')
        day3 =day0-np.timedelta64(3,'D')

        #define the times we will select for VPD
        times_0 = pd.date_range(start=day0, end=day0+np.timedelta64(23,'h'),freq='H')
        print(times_0)
        times_1 = pd.date_range(start=day1, end=day1+np.timedelta64(23,'h'),freq='H')
        times_2 = pd.date_range(start=day2, end=day2+np.timedelta64(23,'h'),freq='H')
        times_3 = pd.date_range(start=day3, end=day3+np.timedelta64(23,'h'),freq='H')
        
        w0 = dat_hrrr['wind_speed'].sel(time=times_0, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd0 = dat_hrrr['vpd_2m'].sel(time=times_0, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd1 = dat_hrrr['vpd_2m'].sel(time=times_1, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd1=hd1.assign_coords({'time':w0['time'].values})
        hd2 = dat_hrrr['vpd_2m'].sel(time=times_2, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd2=hd2.assign_coords({'time':w0['time'].values})
        hd3 = dat_hrrr['vpd_2m'].sel(time=times_3, grid_yt = np.unique(intersection_sub['row'].values),grid_xt = np.unique(intersection_sub['col'].values))
        hd3=hd3.assign_coords({'time':w0['time'].values})
            
        hd0w0 = hd0*w0
        hd1w0 = hd1*w0
        hd2w0 = hd2*w0
        hd3w0 = hd3*w0

        hd0w0_daily_mean = hd0w0.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        hd1w0_daily_mean = hd1w0.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        hd2w0_daily_mean = hd2w0.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        hd3w0_daily_mean = hd3w0.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        

        
        df_hrrr_derived['day'].iloc[count] =today# pd.to_datetime([str(day_start_hour)+ 'Z Start Day'].values[count])
        if use_weights==True:
            df_hrrr_derived.loc[count, ('hd0w0')] = np.nansum((hd0w0_daily_mean.values)*(intersection_sub['weights'].values))
            df_hrrr_derived.loc[count, ('hd1w0')] = np.nansum((hd1w0_daily_mean.values)*(intersection_sub['weights'].values))
            df_hrrr_derived.loc[count, ('hd2w0')] = np.nansum((hd2w0_daily_mean.values)*(intersection_sub['weights'].values))
            df_hrrr_derived.loc[count, ('hd3w0')] = np.nansum((hd3w0_daily_mean.values)*(intersection_sub['weights'].values))

        else:
            df_hrrr_derived.loc[count, ('hd0w0')] = np.nanmean((hd0w0_daily_mean.values))
            df_hrrr_derived.loc[count, ('hd1w0')] = np.nanmean((hd1w0_daily_mean.values))
            df_hrrr_derived.loc[count, ('hd2w0')] = np.nanmean((hd2w0_daily_mean.values))
            df_hrrr_derived.loc[count, ('hd3w0')] = np.nanmean((hd3w0_daily_mean.values))        
        print(df_hrrr_derived)
        count=count+1
    return df_hrrr_derived

In [110]:
""" hd0w0 = dat_hrrr['wind_speed'].sel(time=times_0)*dat_hrrr['vpd_2m'].sel(time=times_0)

        hd0w0_daily_mean= hd0w0.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                           grid_xt = np.unique(intersection_sub['col'].values)).resample(time='24H',
                                                                                      base=day_start_hour, 
                                                                                      label='left').mean(dim='time')

        
        #print(dat_hrrr.sel(time=times_0))
        
        
        dat_hrrr['hd0w0'] = dat_hrrr['wind_speed']*dat_hrrr['vpd_2m']
        #print(dat_hrrr)
        
        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)) #get the location of the overlaps
    
        #TO DO: ADD IF STATEMENT FOR WEIGHTED OR UNWEIGHTED
        if use_weights==True:
            df_hrrr_derived.loc[count, ('hd0w0')] = np.nansum((hrrr_daily_mean_region['hd0w0'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
            df_hrrr_derived.loc[count, ('hwp')] =np.nansum((hrrr_daily_mean_region['hwp'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))   
        else:
            df_hrrr_derived['hd0w0'].iloc[count] = np.nanmean((hrrr_daily_mean_region['hd0w0'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights_mask'].values))
            df_hrrr_derived['hwp'].iloc[count] = np.nanmean((hrrr_daily_mean_region['hwp'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights_mask'].values))

        dat_hrrr.close()
        count =count+1
        #print(df_hrrr_derived)
    return df_hrrr_derived"""

" hd0w0 = dat_hrrr['wind_speed'].sel(time=times_0)*dat_hrrr['vpd_2m'].sel(time=times_0)\n\n        hd0w0_daily_mean= hd0w0.sel(grid_yt = np.unique(intersection_sub['row'].values),\n                                           grid_xt = np.unique(intersection_sub['col'].values)).resample(time='24H',\n                                                                                      base=day_start_hour, \n                                                                                      label='left').mean(dim='time')\n\n        \n        #print(dat_hrrr.sel(time=times_0))\n        \n        \n        dat_hrrr['hd0w0'] = dat_hrrr['wind_speed']*dat_hrrr['vpd_2m']\n        #print(dat_hrrr)\n        \n        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        \n        \n        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),\n                                     

In [2]:
#HRRR_WS formulation from, take time mean, then take weighted average. For HDW, multiply the weighted means of VPD and WIND
def hrrr_timeseries(df,day_start_hour, use_weights):  #with the wind speed
    varis_hrrr_derived = ['day','hwp'] #'hd1w0','hd2w0', 'hd3w0', 'hd4w0', 'hd5w0',
    df_hrrr_derived = generate_df(varis_hrrr_derived, len(df))
    
    #do the intersection, in parallel
    #print(tic)
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'HRRR_GRID',3000) 
                                 for ii in range(len(df)))
    
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        intersection_sub['weights_mask'] =xr.where(intersection_sub['weights']>0,1, np.nan)
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(1,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        print(times_back)
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/processed_hrrr_hdw_hwp/Processed_HRRR_YYYYMMDDHH_HDW_HWP.nc')
        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})
        
        dat_hrrr['hd0w0'] = dat_hrrr['wind_speed']*dat_hrrr['vpd_2m']
        #print(dat_hrrr)
        
        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)) #get the location of the overlaps
    
        df_hrrr_derived['day'].iloc[count] =today# pd.to_datetime([str(day_start_hour)+ 'Z Start Day'].values[count])
        #TO DO: ADD IF STATEMENT FOR WEIGHTED OR UNWEIGHTED
        if use_weights==True:
            df_hrrr_derived.loc[count, ('hd0w0')] = np.nansum((hrrr_daily_mean_region['hd0w0'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
            df_hrrr_derived.loc[count, ('hwp')] =np.nansum((hrrr_daily_mean_region['hwp'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))   
        else:
            df_hrrr_derived['hd0w0'].iloc[count] = np.nanmean((hrrr_daily_mean_region['hd0w0'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights_mask'].values))
            df_hrrr_derived['hwp'].iloc[count] = np.nanmean((hrrr_daily_mean_region['hwp'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights_mask'].values))

        dat_hrrr.close()
        count =count+1
        #print(df_hrrr_derived)
    return df_hrrr_derived

#### Code for calculating LAGGED HDW

In [ ]:
        """#print(hrrr_daily_mean_region['time'].values)
        hd0 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((hrrr_daily_mean_region['wind_speed'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hwp = np.nansum((hrrr_daily_mean_region['hwp'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        """

### ESI (choose weighted or unweighted)

In [34]:
def esi_timeseries(df, day_start_hour):
    #preallocate space for the output
    df_esi_weighted = pd.DataFrame({'day':np.zeros(len(df)),'ESI':np.zeros(len(df))})
    df_esi_unweighted = pd.DataFrame({'day':np.zeros(len(df)),'ESI':np.zeros(len(df))})
    
    
    #do the intersection, in parallel
    esi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df_fire.iloc[ii:ii+1],'ESI_GRID',5000) 
                                 for ii in range(len(df_fire)))
    print([esi_intersections[jj]['weights'].sum() for jj in range(len(esi_intersections))])

    
    fire_esi_intersection=gpd.GeoDataFrame(pd.concat(esi_intersections, ignore_index=True))
    fire_esi_intersection.set_geometry(col='geometry')
    
    
    fire_esi_intersection = fire_esi_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    
    fire_esi_intersection_xr = fire_esi_intersection.to_xarray()
    fire_esi_intersection_xr['weights_mask'] = xr.where(fire_esi_intersection_xr['weights']>0,1, np.nan)
    print(fire_esi_intersection_xr)
    #load in esi data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0])-np.timedelta64(1,'W'),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+np.timedelta64(1,'W')+
                        np.timedelta64(1,'D')) #buffer by a week on each side in case the weekly data timestep isn't a fire day
    esi_filenames, esi_times = make_file_namelist(times,'/data2/lthapa/YYYY/ESI/DFPPM_4WK_YYYYJJJ.nc')
    
    print(esi_filenames)
    print(esi_times)
    
    
    #open the esi files
    dat_esi = xr.open_mfdataset(esi_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_esi = dat_esi.assign_coords({'time': esi_times}) #assign coords so we can resample along time
    dat_esi = dat_esi.where(dat_esi['Band1']!=-9999) #gets rid of the missing values!
    dat_esi_daily = dat_esi.reindex(time=times,method='nearest') #makes the weekly data daily
    dat_esi_daily_sub = dat_esi_daily.sel(lat = fire_esi_intersection_xr['lat'].values, 
                                          lon = fire_esi_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_esi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
                                          

    print(df_esi_unweighted)
    print(fire_esi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    
    df_esi_weighted['day'].iloc[:] = pd.to_datetime(fire_esi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    df_esi_unweighted['day'].iloc[:] = pd.to_datetime(fire_esi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)

    varis=['ESI']
    for var in varis:
        df_esi_weighted[var] = np.nansum(fire_esi_intersection_xr['weights'].values*dat_esi_daily_sub['Band1'].values, axis=(1,2))
        df_esi_unweighted[var] = np.nanmean(fire_esi_intersection_xr['weights_mask'].values*dat_esi_daily_sub['Band1'].values, axis=(1,2))
    
    return df_esi_weighted, df_esi_unweighted
    

### Fuel Loading (990m)

In [50]:
def fuel_loading_timeseries(df, day_start_hour):
    #do the intersection, in parallel
    tic = datetime.datetime.now()
    fuel_fwi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'FUEL_FWI_GRID_990M',2000) 
                                 for ii in range(len(df)))
    toc = datetime.datetime.now()
    print(toc-tic)
    print([fuel_fwi_intersections[jj]['weights'].sum() for jj in range(len(fuel_fwi_intersections))])
    
    fire_fuel_fwi_intersection=gpd.GeoDataFrame(pd.concat(fuel_fwi_intersections, ignore_index=True))
    fire_fuel_fwi_intersection = fire_fuel_fwi_intersection.drop(columns='geometry')
    fire_fuel_fwi_intersection = fire_fuel_fwi_intersection.set_index(['12Z Start Day','row', 'col'])
    fire_fuel_fwi_intersection_xr = fire_fuel_fwi_intersection.to_xarray()
    fire_fuel_fwi_intersection_xr['weights_mask'] = xr.where(fire_fuel_fwi_intersection_xr['weights']>0,1, np.nan)

    path_fuel_fwi = '/data2/lthapa/ML_daily/fuel_fwi_990m.nc'
    dat_fuel_fwi = xr.open_dataset(path_fuel_fwi) #map is fixed in time
    dat_fuel_fwi_daily = dat_fuel_fwi.expand_dims({'time': pd.to_datetime(fire_fuel_fwi_intersection_xr['12Z Start Day'].values)}) #the PWS expanded over all the days

    dat_fuel_fwi_sub_daily = dat_fuel_fwi_daily.sel(row = fire_fuel_fwi_intersection_xr['row'].values, 
                                        col = fire_fuel_fwi_intersection_xr['col'].values, method='nearest')
    print(dat_fuel_fwi_sub_daily)
    #preallocate space for the output
    varis = ['day','Extreme_N', 'VeryHigh_N','High_N', 'Moderate_N', 'Low_N']
    df_loading_weighted = generate_df(varis, len(df))
    df_loading_unweighted = generate_df(varis, len(df))

    df_loading_weighted['day'] = df['12Z Start Day'].values
    df_loading_unweighted['day'] = df['12Z Start Day'].values

    for var in varis[1:len(varis)]:
        df_loading_weighted[var] = np.nansum(fire_fuel_fwi_intersection_xr['weights'].values*dat_fuel_fwi_sub_daily[var].values, axis=(1,2))
        df_loading_unweighted[var] = np.nanmean(fire_fuel_fwi_intersection_xr['weights_mask'].values*dat_fuel_fwi_sub_daily[var].values, axis=(1,2))

    return df_loading_weighted, df_loading_unweighted
    

### FWI IMERG (WEIGHTED VS UNWEIGHTED)

In [23]:
def imerg_fwi_timeseries(df, day_start_hour):
    varis = ['day','IMERG.FINAL.v6_DC','IMERG.FINAL.v6_DMC','IMERG.FINAL.v6_FFMC',
             'IMERG.FINAL.v6_ISI','IMERG.FINAL.v6_BUI','IMERG.FINAL.v6_FWI',
             'IMERG.FINAL.v6_DSR'] 
    df_imerg_weighted = generate_df(varis, len(df))
    df_imerg_unweighted = generate_df(varis, len(df))
    #do the intersection, in parallel
    fwi_intersections = Parallel(n_jobs=1)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'IMERG_FWI_GRID',10000) 
                                 for ii in range(len(df)))
    
    print([fwi_intersections[jj]['weights'].sum() for jj in range(len(fwi_intersections))])

    fire_fwi_intersection=gpd.GeoDataFrame(pd.concat(fwi_intersections, ignore_index=True))
    fire_fwi_intersection.set_geometry(col='geometry')    
    fire_fwi_intersection = fire_fwi_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])

    fire_fwi_intersection=fire_fwi_intersection[~fire_fwi_intersection.index.duplicated()]
    
    fire_fwi_intersection_xr = fire_fwi_intersection.to_xarray()
    fire_fwi_intersection_xr['weights_mask'] = xr.where(fire_fwi_intersection_xr['weights']>0,1, np.nan)
    print(fire_fwi_intersection_xr)
    
    #load in FWI data associated with the fire
    times = pd.date_range(np.datetime64(df['UTC Day'].iloc[0]),
                        np.datetime64(df['UTC Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    fwi_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/YYYY/FWI_IMERG/FWI.IMERG.FINAL.v6.Daily.Default.YYYYMMDD.nc')
    
    dat_fwi = xr.open_mfdataset(fwi_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_fwi = dat_fwi.assign_coords({'time': times_back_used}) #assign coords so we can resample along time
    dat_fwi = dat_fwi.resample(time='1H').pad() #make the data hourly, so we can define the day as 12z-12z instead of 0z-0z
    dat_fwi_mean = dat_fwi.resample(time='24H',base=day_start_hour ,label='left').mean(dim='time') #take the daily mean         
    
    #select the locations and times we want
    dat_fwi_sub = dat_fwi_mean.sel(lat = fire_fwi_intersection_xr['lat'].values, 
                                    lon = fire_fwi_intersection_xr['lon'].values).sel(
                            time = pd.to_datetime(fire_fwi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+ ' '+
                                         str(day_start_hour)+':00:00'), method='nearest')#these should be lined up correctly

    print(dat_fwi_sub)
    df_imerg_weighted['day'].iloc[:] = pd.to_datetime(fire_fwi_intersection_xr['12Z Start Day'].values)
    df_imerg_unweighted['day'].iloc[:] = pd.to_datetime(fire_fwi_intersection_xr['12Z Start Day'].values)

    for var in varis[1:len(varis)]:
        df_imerg_weighted[var] = np.nansum(fire_fwi_intersection_xr['weights'].values*dat_fwi_sub[var].values, axis=(1,2)) #weighted average
        df_imerg_unweighted[var] = np.nanmean(fire_fwi_intersection_xr['weights_mask'].values*dat_fwi_sub[var].values,axis=(1,2)) #mask and average
    return df_imerg_weighted, df_imerg_unweighted

### PWS (NEED TO FIX ONCE WE DECIDE WEIGHTED VS UNWEIGHTED)

In [36]:
def pws_timeseries(df, day_start_hour):
    #do the intersection, in parallel
    tic = datetime.datetime.now()
    pws_intersections = Parallel(n_jobs=1)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'PWS_GRID',4000) 
                                 for ii in range(len(df)))
    toc = datetime.datetime.now()
    print(toc-tic)
    print([pws_intersections[jj]['weights'].sum() for jj in range(len(pws_intersections))])

    
    fire_pws_intersection=gpd.GeoDataFrame(pd.concat(pws_intersections, ignore_index=True))
    fire_pws_intersection.set_geometry(col='geometry')
    fire_pws_intersection = fire_pws_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    
    fire_pws_intersection_xr = fire_pws_intersection.to_xarray()
    
    #nc["cdd_hdd"] = xr.where(nc["tavg"] > 65, nc["tavg"] - 65, 65 - nc["tavg"])
    fire_pws_intersection_xr['weights_mask'] = xr.where(fire_pws_intersection_xr['weights']>0,1, np.nan)
    
    #load in PWS data associated with the fire (it's only one dataset)  
    #open the PWS files
    path_pws = '/data2/lthapa/PWS_6_jan_2021.nc'
    dat_pws = xr.open_dataset(path_pws) #map is fixed in time
    #print(dat_pws)
    
    dat_pws = dat_pws.assign_coords({'time': pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)})
    dat_pws_daily = dat_pws['Band1'].expand_dims({'time': pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)}) #the PWS expanded over all the days
    
    dat_pws_daily_sub = dat_pws_daily.sel(lat = fire_pws_intersection_xr['lat'].values, 
                                          lon = fire_pws_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
    ndays = len(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    df_pws_weighted = pd.DataFrame({'day':np.zeros(ndays),'PWS':np.zeros(ndays)})
    df_pws_unweighted = pd.DataFrame({'day':np.zeros(ndays),'PWS':np.zeros(ndays)})


    df_pws_weighted['day'].iloc[:] = pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    df_pws_unweighted['day'].iloc[:] = pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)

    varis=['PWS']
    for var in varis:
        df_pws_weighted[var] = np.nansum(fire_pws_intersection_xr['weights'].values*dat_pws_daily_sub.values, axis=(1,2)) #WEIGHTED AVERAGE
        df_pws_unweighted[var] = np.nanmean(fire_pws_intersection_xr['weights_mask'].values*dat_pws_daily_sub.values, axis=(1,2)) #MASK AND AVERAGE
        #print(np.nanmean(dat_pws_daily_sub.values, axis=(1,2)))
        #df_pws[var] = dat_pws_daily_sub.mean(dim=['lat','lon'], skipna=True)
    return df_pws_weighted, df_pws_unweighted
    

### SMOPS Data (need to fix once we decide on weighted vs unweighted)

In [45]:
def smops_timeseries(df, day_start_hour, use_weights):
    varis_smops = ['day','Blended_SM']
    df_smops = generate_df(varis_smops, len(df))
    #do the intersection, in parallel
    smops_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'SMOPS_GRID',25000) 
                                 for ii in range(len(df)))
    print([smops_intersections[jj]['weights'].sum() for jj in range(len(smops_intersections))])

    fire_smops_intersection=gpd.GeoDataFrame(pd.concat(smops_intersections, ignore_index=True))
    fire_smops_intersection.set_geometry(col='geometry')  
    fire_smops_intersection = fire_smops_intersection.set_index([str(day_start_hour)+'Z Start Day', 'row', 'col'])
    fire_smops_intersection=fire_smops_intersection[~fire_smops_intersection.index.duplicated()]

    fire_smops_intersection_xr = fire_smops_intersection.to_xarray()
    fire_smops_intersection_xr['weights_mask'] = xr.where(fire_smops_intersection_xr['weights']>0,1, np.nan)
    #print(fire_smops_intersection_xr['weights_mask'])

    #load in rave data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    smops_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/YYYY/SMOPS/NPR_SMOPS_CMAP_DYYYYMMDD.nc')

    dat_smops = xr.open_mfdataset(smops_filenames,concat_dim='Time',combine='nested',compat='override', coords='all')
    dat_smops = dat_smops.assign_coords({'Time': times_back_used}) #assign coords so we can select in time
    #print(dat_smops['Time'].values)
    #print(fire_smops_intersection_xr[str(day_start_hour)+'Z Start Day'].values)
    #select the locations and times we want
    dat_smops_sub = dat_smops.isel(Latitude = fire_smops_intersection_xr['row'].values.astype(int), 
                    Longitude = fire_smops_intersection_xr['col'].values.astype(int)).sel(
                    Time = pd.to_datetime(fire_smops_intersection_xr[str(day_start_hour)+'Z Start Day'].values))#these should be lined up correctly

    df_smops['day'].iloc[:] = pd.to_datetime(fire_smops_intersection_xr[str(day_start_hour)+'Z Start Day'].values)
    for var in varis_smops[1:]:
        dat_smops_sub[var]=dat_smops_sub[var].where(dat_smops_sub[var] != -0.0999) #mask out the ocean, there is no soil moisture here
        if use_weights==True:
            df_smops[var] = np.nansum(fire_smops_intersection_xr['weights'].values*dat_smops_sub[var].values, axis=(1,2))
        else:
            df_smops[var] = np.nanmean(fire_smops_intersection_xr['weights_mask'].values*dat_smops_sub[var].values, axis=(1,2)) #MASK AND AVERAGE
    
    return df_smops

### RAVE (3KM)

In [58]:
def rave_timeseries(df, day_start_hour, sum_interval, use_weights):
    varis = ['day','FRP_MEAN']#, 'FRP_SD', 'FRE']#, 'CO2', 'CO', 'SO2', 'OC','BC', 'PM25', 'NOx', 'NH3','TPM', 'VOCs', 'CH4'] #don't need 'area', it's the area of each cell
    df_rave = generate_df(varis, len(df))

    #do the intersection, in parallel
    rave_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'RAVE_GRID_3KM',3000) 
                                 for ii in range(len(df)))
    print([rave_intersections[jj]['weights'].sum() for jj in range(len(rave_intersections))])

    fire_rave_intersection=gpd.GeoDataFrame(pd.concat(rave_intersections, ignore_index=True))
    fire_rave_intersection.set_geometry(col='geometry')    
    #print(fire_rave_intersection)
    fire_rave_intersection = fire_rave_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    fire_rave_intersection=fire_rave_intersection[~fire_rave_intersection.index.duplicated()]

    fire_rave_intersection_xr = fire_rave_intersection.to_xarray()
    fire_rave_intersection_xr['weights_mask'] = xr.where(fire_rave_intersection_xr['weights']>0,1, np.nan)

    #load in rave data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    rave_filenames,times_back_used = make_file_namelist(times,
                                                        '/data2/lthapa/YYYY/RAVE/MM/RAVE-HrlyEmiss-3km-CONUS_v1r1_blend_sYYYYMMDD.nc')                                                 
    
    #print(rave_filenames)
    dat_rave = xr.open_mfdataset(rave_filenames,concat_dim='time',combine='nested',compat='override', coords='all')

    dat_rave = dat_rave.resample(time=str(sum_interval)+'H',base=day_start_hour).sum(dim='time') #take the daily sum
    
    #select the locations and times we want
    dat_rave_sub = dat_rave.isel(grid_yt = fire_rave_intersection_xr['row'].values.astype(int), 
                    grid_xt = fire_rave_intersection_xr['col'].values.astype(int)).sel(
                    time = pd.to_datetime(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                         'T12:00:00.000000000'))#these should be lined up correctly
    ndays = len(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'])

    df_rave['day'].iloc[:] = pd.to_datetime(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    for var in varis[1:]:
        if use_weights==True:
            df_rave[var] = np.nansum(fire_rave_intersection_xr['weights'].values*dat_rave_sub[var].values, axis=(1,2))
        else:
            df_rave[var] = np.nansum(fire_rave_intersection_xr['weights_mask'].values*dat_rave_sub[var].values,axis=(1,2))
    
    return df_rave

### NCAR Moisture Data (DON'T WORRY ABOUT THIS FOR NOW)

## Dataset-Independent Helper Functions

In [59]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices

def build_one_gridcell(LAT_COR, LON_COR, LAT_CTR, LON_CTR, loc):
    ii=loc[0]
    jj=loc[1]
    #corners of the grid box
    sw = (LON_COR[ii, jj],LAT_COR[ii, jj]) #SW
    se =(LON_COR[ii, jj+1],LAT_COR[ii, jj+1]) #SE
    nw = (LON_COR[ii+1, jj],LAT_COR[ii+1, jj]) #NW
    ne = (LON_COR[ii+1, jj+1],LAT_COR[ii+1, jj+1]) #NE
    #polygon of the grid box        
    poly_cell = Polygon([sw,nw,ne,se])
    return LAT_CTR[ii,jj], LON_CTR[ii,jj],ii,jj,poly_cell

In [150]:
#poly is the polygon for one timestep (in lambert conformal conic)
#dataset_name is the name of a model grid nc file
#bf is the size of the polygon buffer

def calculate_intersection(poly,dataset_name,bf):
    print(poly['12Z Start Day'])
    #load in the merra grid
    grid = xr.open_dataset(dataset_name+'.nc')
    
    #get the bounds of the buffered polygons
    #poly_latlon =poly#.to_crs(epsg=4326)
    #bounds = poly_latlon.buffer(bf).bounds
    bounds = poly.buffer(bf).to_crs(epsg=4326).bounds
    print(bounds)
    #first check for rows and cols, filtering near the polygon
    [rows,cols] = np.where((grid.LAT_CTR.values>bounds['miny'].values)&
                    (grid.LAT_CTR.values<bounds['maxy'].values)&
                    (grid.LON_CTR.values>bounds['minx'].values)&
                    (grid.LON_CTR.values<bounds['maxx'].values))
    locs = zip(rows,cols)


    #make a geodataframe (in paralell of the rows and cols)
    results = Parallel(n_jobs=8)(delayed(build_one_gridcell)
                                 (grid['LAT_COR'].values, grid['LON_COR'].values,
                                  grid['LAT_CTR'].values, grid['LON_CTR'].values,loc) 
                                 for loc in locs)

    #format the grid subset into a dataframs
    df_grid=gpd.GeoDataFrame(results)
    df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
    df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:4326') #need to say it's in lat/lon before transform to LCC
    df_grid=df_grid.to_crs(epsg=3347)

    #intersect the polygon with the grid subset
    intersection = gpd.overlay(df_grid, poly, how='intersection',keep_geom_type=False).drop_duplicates()
    intersection['grid intersection area (ha)'] =intersection['geometry'].area/10000
    intersection['weights'] = intersection['grid intersection area (ha)']/intersection['fire area (ha)'] 
    
    return intersection

In [61]:
#LAT and LON are 2d arrays
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LAT[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    lon_corners = (LON[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LON[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    return lat_corners, lon_corners


In [62]:
def make_file_namelist(time,base_filename):
    filename_list = np.array([])
    times_back_used = np.array([])
    for jj in range(len(time)):
        fname = base_filename.replace('YYYY',time[jj].strftime('%Y')).\
                                replace('MM',time[jj].strftime('%m')).\
                                replace('DD',time[jj].strftime('%d')).\
                                replace('HH',time[jj].strftime('%H')).\
                                replace('JJJ',time[jj].strftime('%j'))
        #print(fname)
        if exists(fname):
            filename_list = np.append(filename_list,fname)
            times_back_used = np.append(times_back_used,time[jj])
    return filename_list, times_back_used

In [63]:
def generate_df(variables, length):
    df = pd.DataFrame()
    for vv in variables:
        df[vv] = np.zeros(length)
    return df